In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html, Output, Input
import dash_bootstrap_components as dbc
from datetime import datetime
#solo para google colab
#!pip install jupyter-dash

In [2]:
df=pd.read_csv('Tablero_solicitudes_check.csv',delimiter=';')

## Exploración de datos

In [3]:
df.columns

Index(['Instancia', 'Matrícula', 'Nombre', 'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       'OportunidadesSeleccionadas', 'OportunidadesAprobadas', 'Estatus',
       'OportunidadAsignada', 'Nombre Oportunidad Asignada',
       'CampusAdministrador', 'PeriodoAcadémico', 'Actividad Actual', 'Fecha'],
      dtype='object')

In [4]:
fig = px.histogram(df, x="Promedio")
fig.show()

In [5]:
fig = px.histogram(df, x="Núm. MatAprobadas")
fig.show()

In [6]:
fig = px.histogram(df, x="Núm. MatTotales")
fig.show()

## Limpieza de datos

In [7]:
df = df.drop(['OportunidadesAprobadas','Actividad Actual'],axis=1)

In [49]:
df.Estatus.unique()

array(['Asignada', 'Aprobada', 'Pre-asignado', 'Rechazada',
       'Asignada - Pendiente registrar materias',
       'Admitido - Documentos de Salida',
       'Admitido - Documentos aprobados', 'Admitido',
       'Asignada - Pendiente enviar documentos',
       'En extranjero - Pendiente registrar materias',
       'Asignada - Materias aprobadas',
       'En extranjero - Materias aprobadas',
       'Asignada - Materias en aprobación',
       'En extranjero - Materias en aprobación',
       'Asignada - Materias enviadas a aprobación', 'Enviado a',
       'En extranjero - Materias enviadas a aprobación',
       'En extranjero - Materias rechazadas (1)',
       'Asignada - Materias rechazadas (2)',
       'Asignada - Documentos enviados a aprobación',
       'En extranjero - Regreso enviado', 'Acreditación en proceso',
       'Asignada - Documentos aprobados',
       'Admitido - Documentos enviados a aprobación',
       'Asignada - En espera de admisión',
       'En extranjero - Materias 

In [54]:
asignados= df[df.Estatus.isin(['Asignada','Asignado','Aprobada'])]

In [55]:
asignados.isna().sum()

Instancia                         0
Matrícula                         0
Nombre                            1
Ap Paterno                        1
Ap Materno                      114
Campus                           40
Programa                          2
Promedio                       2389
Núm. MatAprobadas                 1
Núm. MatTotales                 418
PuntajeAsignación                10
Nivel                             1
Periodo Solicitud                 0
OportunidadesSeleccionadas        0
Estatus                           0
OportunidadAsignada              20
Nombre Oportunidad Asignada       0
CampusAdministrador              20
PeriodoAcadémico                 20
Fecha                             0
dtype: int64

In [56]:
asignados['Ap Materno'].fillna('',inplace=True)

C:\Users\Propietario\AppData\Local\Temp\ipykernel_9488\2741969678.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
datos = asignados.copy()
datos = datos.dropna()

In [131]:
datos[datos.Nivel != 'Indefinido']['Nivel'].unique()

array(['Profesional', 'Maestría', 'Preparatoria', 'Doctorado'],
      dtype=object)

In [58]:
primera=[]
for index in range(len(datos)):
    substring = datos['OportunidadesSeleccionadas'].iloc[index].partition('1 - ')[2]
    opcion = substring.split(',')[0]
    primera.append(opcion)

datos['1era Opcion'] = primera
    

In [59]:
carreras = []
import re
for index in range(len(datos)):
    valor = re.split('\d',datos['Programa'].iloc[index])[0]
    valor = valor.strip()
    carreras.append(valor)

datos['Carrera']=carreras

In [60]:
datos['Carrera']

0        INT
1        ITC
2        ISC
3        CPF
4        LMC
        ... 
81321    ARQ
81329    LIN
81330    LAE
81349    LDI
81352    IFI
Name: Carrera, Length: 9263, dtype: object

In [61]:
datos.columns

Index(['Instancia', 'Matrícula', 'Nombre', 'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       'OportunidadesSeleccionadas', 'Estatus', 'OportunidadAsignada',
       'Nombre Oportunidad Asignada', 'CampusAdministrador',
       'PeriodoAcadémico', 'Fecha', '1era Opcion', 'Carrera'],
      dtype='object')

In [62]:
datos['En 1era']= np.where(datos['1era Opcion']==datos['OportunidadAsignada'],1,0)

In [63]:
areas_carrera=pd.read_csv('Programas_Tec.csv',encoding='utf8')

In [64]:
test = datos.merge(areas_carrera,how='left',left_on='Programa',right_on='Clave')
test= test.drop(['Area Academica','Clave','Programa Academico'],axis=1)

In [65]:
regiones = pd.read_csv('Regiones_Campus.csv')

In [66]:
test = test.merge(regiones,'left','Campus')

In [67]:
def p_int(df):
    for i in range(len(df)):
        value = df['OportunidadAsignada']
        x = re.findall("INT-", value)
        if len(x) == 1:
            return 'INT'
        else:
            return 'SA'

In [68]:
test['P_Int']= test.apply(p_int,axis=1)

In [69]:
test.columns

Index(['Instancia', 'Matrícula', 'Nombre', 'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       'OportunidadesSeleccionadas', 'Estatus', 'OportunidadAsignada',
       'Nombre Oportunidad Asignada', 'CampusAdministrador',
       'PeriodoAcadémico', 'Fecha', '1era Opcion', 'Carrera', 'En 1era',
       'Escuela', 'Region', 'P_Int'],
      dtype='object')

In [70]:
test['Fecha'] = pd.to_datetime(test['Fecha'])

In [71]:
fig = px.histogram(test, x='Region',color='P_Int')
fig.show()

In [72]:
fig = px.pie(test, values='Instancia', names='P_Int', title='Comparativa Intercambio - Study Abroad')
fig.show()

In [73]:
#fig = px.parallel_categories(test,dimensions= ['SizeDog','AnimalGender','AnimalBirthYear'], color="AnimalBirthYear",labels={'SizeDog':'Size','AnimalGender':'Gender','AnimalBirthYear':'BirthYear'})

In [74]:
test['Fecha'] = pd.to_datetime(test["Fecha"].dt.strftime('%Y-%m-%d'))

In [75]:
test.columns

Index(['Instancia', 'Matrícula', 'Nombre', 'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       'OportunidadesSeleccionadas', 'Estatus', 'OportunidadAsignada',
       'Nombre Oportunidad Asignada', 'CampusAdministrador',
       'PeriodoAcadémico', 'Fecha', '1era Opcion', 'Carrera', 'En 1era',
       'Escuela', 'Region', 'P_Int'],
      dtype='object')

In [76]:
ts= test['Fecha'].min().to_pydatetime()
ts

datetime.datetime(2014, 1, 1, 0, 0)

In [77]:
freq_INT = test[test.P_Int == 'INT'].groupby(test['Fecha'].astype('datetime64[M]'))['Instancia'].count().rename('Count').to_frame()
freq_SA = test[test.P_Int == 'SA'].groupby(test['Fecha'].astype('datetime64[M]'))['Instancia'].count().rename('Count').to_frame()
# freq_date = test.groupby(test['Fecha'].astype('datetime64[M]'))['Instancia'].count().rename('Count').to_frame()
freq_INT['Tipo'] = 'INT'
freq_SA['Tipo'] = 'SA'
freq_tipo = pd.concat([freq_INT, freq_SA])


In [78]:
fig = px.line(freq_tipo,y='Count', title='Personas aprobadas a traves del tiempo',color='Tipo')
fig.show()


In [79]:
freq_INT = test[test.P_Int == 'INT'].groupby('Campus')['Instancia'].count().rename('Count_INT').to_frame()
freq_SA = test[test.P_Int == 'SA'].groupby('Campus')['Instancia'].count().rename('Count_SA').to_frame()

merged = freq_INT.merge(freq_SA,on = 'Campus')
merged['Total'] = merged['Count_INT']+merged['Count_SA']
merged['%INT'] = (merged['Count_INT']/(merged['Count_INT']+merged['Count_SA'])) *100
merged.reset_index(inplace=True)
fig = px.scatter(merged,x='Campus',y='%INT')
fig.show()

In [80]:
test.columns

Index(['Instancia', 'Matrícula', 'Nombre', 'Ap Paterno', 'Ap Materno',
       'Campus', 'Programa', 'Promedio', 'Núm. MatAprobadas',
       'Núm. MatTotales', 'PuntajeAsignación', 'Nivel', 'Periodo Solicitud',
       'OportunidadesSeleccionadas', 'Estatus', 'OportunidadAsignada',
       'Nombre Oportunidad Asignada', 'CampusAdministrador',
       'PeriodoAcadémico', 'Fecha', '1era Opcion', 'Carrera', 'En 1era',
       'Escuela', 'Region', 'P_Int'],
      dtype='object')

In [104]:
df_query.dtypes

Region              object
Campus              object
Fecha       datetime64[ns]
Promedio           float64
dtype: object

In [120]:
df_query = pd.DataFrame(test.groupby(['Region','Campus'])['Promedio'].mean())

df_query= df_query.reset_index()
df_query.sort_values(['Region','Campus'])


,Region,Campus,Promedio
0,Centro Sur,Cuernavaca,87.668902
1,Centro Sur,Hidalgo,86.831917
2,Centro Sur,Puebla,87.587732
3,Centro Sur,Querétaro,88.783709
4,Centro Sur,San Luis Potosí,87.633500
5,Centro Sur,Tampico,86.441509
6,Centro Sur,Toluca,88.525309
7,Ciudad de Mexico,Ciudad de México,87.606319
8,Ciudad de Mexico,EGADE Ciudad de Mexico,91.999500
9,Ciudad de Mexico,EGADE Santa Fe,92.873623


In [124]:
datos.Nivel.unique()

array(['Profesional', 'Maestría', 'Preparatoria', 'Doctorado',
       'Indefinido'], dtype=object)

In [128]:
datos

8817    0
Name: Programa, dtype: object

In [121]:
fig = px.scatter(df_query, y=['Region','Campus'], x="Promedio", color="Region", symbol="Region",labels={'value':'Campus'})
fig.update_yaxes(tickvals=df_query.Campus.unique(),ticklen=6)
fig.update_layout(height= 900,width =1000)
fig.show()
